<h1 align=center><font size = 8>Capstone Project: The Battle of Neighbourhoods</font></h1>

<h2 align=left><font size = 6>1. Introduction</font></h2>

<h3 align=left><font size = 5>1.1 Description of the Problem</font></h3>

There are many practical questions that require the comparison across city neighborhoods. For example, a job seeker with transferable skills may wish to focus his/her search on a single neighborhood with jobs that best match his/her qualifications, rather than dispersing his/her search efforts across multiple neighborhoods. Likewise, a restaurant looking to expand its locations might perhaps select neighborhoods it wishes to expand into before considering particular sites or neighborhoods. Additionally, many within-city computations might be aided by modelling a neighborhood’s relationship to other neighborhoods. For example, a person buying or renting a home in a new city might want to be able to compare the neighborhoods of the city.

<h3 align=left><font size = 5>1.2 Objective</font></h3>
Taipei and New Taipei City are two major cities in Taiwan. Both cities have been centers of attention for residential, job employment, tourism, education, shopping and sport activities. Both municipalities located in the north of Taiwan with New Taipei City surrounds Taipei. Taipei is the capital city of Taiwan. Most part of New Taipei City are mountain area. The aim of this project is to segment these two cities' neighborhoods based on data collected from Foursquare about venue categories in totally 41 neighborhoods across the cities. Using segmentation and clustering, I hope I can determine:  


1. The similarity or dissimilarity between neighborhoods.
2. Classification of a neighborhood inside a city whether it is residential, tourism places, or others.

<h2 align=left><font size = 6>2. Data</font></h2>

<h3 align=left><font size = 5>2.1 Description of Data</font></h3>

This project will rely on public data from websites and Foursquare.

In order to segment the neighborhoods and explore them, we will essentially need a dataset that contains the 2 boroughs and the neighborhoods that exist in each borough as well as the latitude and longitude coordinates of each neighborhood. 

A Wikipedia page: https://en.wikipedia.org/wiki/List_of_townships,_county-administered_cities_and_districts_of_Taiwan exists that has a list of townships, county-administered cities and districts of Taiwan. We will have to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format that consist of two columns of 2 Boroughs and 41 Neighborhoods of both cities.

We will then use the geopy library to get the latitude and longitude values of all neighborhoods.

<h1 align=left><font size = 5>2.2 Download, scrape and wrangle</font></h1>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup
import lxml
print('Libraries imported.')

Libraries imported.


Now, let's retrieve neighborhood names from the Wikipedia page and fill the dataframe with borough and neighborhood data.

In [2]:
# Download data and parse it:
results = requests.get('https://en.wikipedia.org/wiki/List_of_townships,_county-administered_cities_and_districts_of_Taiwan')
soup = BeautifulSoup(results.text, 'html.parser')
table=soup.find('table', class_='wikitable sortable')

# Find all items and skip first one:
rows=table.findAll('tr')
rows=rows[1:len(rows)]
data=rows[2].findAll('a')
name_list=[]

# Retrieve the borough and neighborhood data:
for i, row in enumerate(rows):
    rows[i]=rows[i].findAll('a')
    name_list.append([rows[i][-1].text, rows[i][0].text.replace(' District', '')])

# make dataframe:
name_data=pd.DataFrame(name_list)
name_data.columns=['Borough', 'Neighborhood']

# We only need the Taipei and New Taipei City boroughs and neighborhoods:
name_data=name_data[(name_data['Borough'] == 'Taipei') | (name_data['Borough'] == 'New Taipei City')].reset_index(drop=True)

In [3]:
name_data

,Borough,Neighborhood
0,Taipei,Zhongzheng
1,Taipei,Datong
2,Taipei,Zhongshan
3,Taipei,Songshan
4,Taipei,Da'an
5,Taipei,Wanhua
6,Taipei,Xinyi
7,Taipei,Shilin
8,Taipei,Beitou
9,Taipei,Neihu


Quickly examine the resulting dataframe.

And make sure that the dataset has all 2 boroughs and 41 neighborhoods.

In [4]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(name_data['Borough'].unique()),
        name_data.shape[0]
    )
)

print('The boroughs are: ', set(name_data['Borough']))

The dataframe has 2 boroughs and 41 neighborhoods.
The boroughs are:  {'New Taipei City', 'Taipei'}


### A. City Data

The next task is essentially transforming Geo-spatial data into the neighborhood name dataframe.

#### Use geopy library to get the latitude and longitude values of all neighborhoods.

In order to define an instance of the geocoder, we need to define an user_agent. We will name our agent <em>taipei_explorer</em>, as shown below. Then let's loop through all neighborhoods to retrieve their Geo-spatial data and fill the dataframe with data.

In [5]:
geo_list=[]
for borough, neighborhood in zip(name_data['Borough'], name_data['Neighborhood']):
    address = neighborhood + ', ' + borough

    geolocator = Nominatim(user_agent="taipei_explorer")
    location = geolocator.geocode(address)
    geo_list.append([neighborhood, location.latitude, location.longitude])
    
# Fill the dataframe with data
geo_df=pd.DataFrame(geo_list)
geo_df.columns = ['Neighborhood', 'Latitude', 'Longitude']
geo_data=name_data
geo_data=geo_data.join(geo_df.set_index('Neighborhood'), on='Neighborhood')
geo_data

,Borough,Neighborhood,Latitude,Longitude
0,Taipei,Zhongzheng,25.047565,121.516946
1,Taipei,Datong,25.065986,121.515514
2,Taipei,Zhongshan,25.064361,121.533468
3,Taipei,Songshan,25.049885,121.577272
4,Taipei,Da'an,25.026515,121.534395
5,Taipei,Wanhua,25.031933,121.499332
6,Taipei,Xinyi,25.037520,121.563680
7,Taipei,Shilin,25.091840,121.524207
8,Taipei,Beitou,25.132419,121.501379
9,Taipei,Neihu,25.069664,121.588998


Save location data into csv file.

In [6]:
geo_data.to_csv('TaipeiNeighborhoods.csv', index = False)

### B. City Map

#### Use geopy library to get the latitude and longitude values of Taipei City.

In [7]:
address = 'Taipei, Taiwan'

geolocator = Nominatim(user_agent="taipei_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Taipei City are {}, {}.'.format(latitude, longitude))
print('There are {} neighborhoods in Taipei City and New Taipei City.'.format(geo_data.shape[0]))

The geograpical coordinate of Taipei City are 25.0375198, 121.5636796.
There are 41 neighborhoods in Taipei City and New Taipei City.


#### Create a map of Taipei City and New Taipei City with neighborhoods superimposed on top.

In [8]:
# create map of Taipei City and New Taipei City using latitude and longitude values
map_taipei = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(geo_data['Latitude'], geo_data['Longitude'], geo_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_taipei)  
    
map_taipei

<h2 align=left><font size = 6>3. Methodology</font></h2>

We will utilize the Foursquare API to explore the neighborhoods and then run *k*-means clustering to cluster the neighborhoods into 5 clusters.

<h3 align=left><font size = 5>3.1 Data Exploration</font></h3>

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods.

#### Define Foursquare Credentials, Version, Limit and Radius

In [9]:
CLIENT_ID = 'UDPW2GMS3GJZ2TCEL4YLNDDRPBNJXVIDZ4BL3I1OCZH2U5NM' # your Foursquare ID
CLIENT_SECRET = 'HKJGHFQTQSZYPTFKEJD0MS2LH5MLKTXXRKJVO3IDIEJOWIXX' # your Foursquare Secret
VERSION = '20190609' # Foursquare API version
LIMIT = 100
radius = 2000

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UDPW2GMS3GJZ2TCEL4YLNDDRPBNJXVIDZ4BL3I1OCZH2U5NM
CLIENT_SECRET:HKJGHFQTQSZYPTFKEJD0MS2LH5MLKTXXRKJVO3IDIEJOWIXX


#### Let's create a function to repeat the same venue exploration process for all neighborhoods in both cities.

In [10]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
         
        # make the GET request
        try:
            results = []
            results = requests.get(url).json()["response"]['groups'][0]['items']
        except:
            venues_list.append([(
                name, 
                lat, 
                lng, 
                'NaN', 
                'NaN', 
                'NaN', 
                'NaN')])
        finally:
            # return only relevant information for each nearby venue
            if len(results):    # More than 1 venue
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    v['venue']['name'], 
                    v['venue']['location']['lat'], 
                    v['venue']['location']['lng'],  
                    v['venue']['categories'][0]['name']) for v in results])
            else:               # No venue
                venues_list.append([(
                    name, 
                    lat, 
                    lng, 
                    'NaN', 
                    'NaN', 
                    'NaN', 
                    'NaN')])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *all_venues*.

In [11]:
all_venues = getNearbyVenues(names=geo_data['Neighborhood'],
                                   latitudes=geo_data['Latitude'],
                                   longitudes=geo_data['Longitude']
                                  )

Zhongzheng
Datong
Zhongshan
Songshan
Da'an
Wanhua
Xinyi
Shilin
Beitou
Neihu
Nangang
Wenshan
Banqiao
Xinzhuang
Zhonghe
Yonghe
Tucheng
Shulin
Sanxia
Yingge
Sanchong
Luzhou
Wugu
Taishan
Linkou
Bali
Tamsui
Sanzhi
Shimen
Jinshan
Wanli
Xizhi
Ruifang
Gongliao
Pingxi
Shuangxi
Xindian
Shenkeng
Shiding
Pinglin
Wulai


#### Let's check the size of the resulting dataframe

In [12]:
print('There are {} unique categories.'.format(all_venues.shape[0]))
all_venues.head()

There are 1823 unique categories.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Zhongzheng,25.047565,121.516946,樂盒子 La Caja De Musica,25.050126,121.518196,Spanish Restaurant
1,Zhongzheng,25.047565,121.516946,吉豚屋 Katsuya,25.046016,121.516977,Donburi Restaurant
2,Zhongzheng,25.047565,121.516946,Wilbeck Cafe (威爾貝克手烘咖啡館),25.044778,121.515598,Coffee Shop
3,Zhongzheng,25.047565,121.516946,Coffee Alley (咖啡弄),25.045518,121.515110,Café
4,Zhongzheng,25.047565,121.516946,好滴 Drip Cafe,25.045712,121.516044,Café


#### Let's find out how many unique categories can be curated from all the returned venues

In [13]:
print('There are {} unique categories.'.format(len(all_venues['Venue Category'].unique())))

There are 205 unique categories.


Let's check how many venues were returned for each neighborhood

In [14]:
all_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bali,8,8,8,8,8,8
Banqiao,74,74,74,74,74,74
Beitou,63,63,63,63,63,63
Da'an,100,100,100,100,100,100
Datong,100,100,100,100,100,100
Gongliao,2,2,2,2,2,2
Jinshan,18,18,18,18,18,18
Linkou,10,10,10,10,10,10
Luzhou,22,22,22,22,22,22


<h3 align=left><font size = 5>3.2 Analyze Each Neighborhood</font></h3>

In [15]:
# one hot encoding
all_onehot = pd.get_dummies(all_venues[['Venue Category']], prefix="", prefix_sep="")


# Drop 'no venue' category
# all_onehot.drop('NaN', 1, inplace=True)

# add neighborhood column back to dataframe
all_onehot['Neighborhood'] = all_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [all_onehot.columns[-1]] + list(all_onehot.columns[:-1])
all_onehot = all_onehot[fixed_columns]

all_onehot.head()

,Neighborhood,ATM,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beijing Restaurant,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Building,Burger Joint,Bus Station,Bus Stop,Cable Car,Cafeteria,Café,Campground,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cultural Center,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Historic Site,History Museum,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Night Market,Nightclub,Noodle House,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Train,Train Station,Tunnel,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfall,Women's Store,Zoo,Zoo Exhibit
0,Zhongzheng,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Zhongzheng,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Zhongzheng,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

And let's examine the new dataframe size.

In [16]:
all_onehot.shape

(1823, 206)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [17]:
all_grouped = all_onehot.groupby('Neighborhood').mean().reset_index()
all_grouped.head()

,Neighborhood,ATM,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beijing Restaurant,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Building,Burger Joint,Bus Station,Bus Stop,Cable Car,Cafeteria,Café,Campground,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cultural Center,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Historic Site,History Museum,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Night Market,Nightclub,Noodle House,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Train,Train Station,Tunnel,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfall,Women's Store,Zoo,Zoo Exhibit
0,Bali,0.000000,0.0,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.00,0.125,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.125000,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.125,0.000000,0.125000,0.0,0.0,0.0,0.125000,0.125000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.125000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.125,0.0,0.000000,0.0,0.0,0.0,0.

In [18]:
all_merged_data = geo_data

all_merged_data = all_merged_data.join(all_grouped.set_index('Neighborhood'), on='Neighborhood')
all_merged_data.head()

,Borough,Neighborhood,Latitude,Longitude,ATM,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beijing Restaurant,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Building,Burger Joint,Bus Station,Bus Stop,Cable Car,Cafeteria,Café,Campground,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cultural Center,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Historic Site,History Museum,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Night Market,Nightclub,Noodle House,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Train,Train Station,Tunnel,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfall,Women's Store,Zoo,Zoo Exhibit
0,Taipei,Zhongzheng,25.047565,121.516946,0.0,0.01,0.00,0.0,0.02,0.00,0.02,0.03,0.0,0.00,0.0,0.03,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.00,0.0,0.02,0.0,0.0,0.00,0.0,0.00,0.01,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.09,0.0,0.01,0.00,0.02,0.03,0.0,0.01,0.0,0.05,0.0,0.00,0.01,0.01,0.0,0.0,0.0,0.0,0.0,0.00,0.02,0.0,0.00,0.00,0.0,0.01,0.00,0.01,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.01,0.00,0.0,0.0,0.01,0.00,0.01,0.0,0.00,0.0,0.0,0.01,0.00,0.0,0.0,0.0,0.0,0.01,0.00,0.00,0.00,0.0,0.01,0.0,0.01,0.0,0.0,0.00,0.03,0.0,0.09,0.01,0.03,0.00,0.02,0.00,0.01,0.0,0.0,0.00,0.02,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.00,0.06,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,0.00,0.03,0.02,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.01,0.0,0.01,0.0,0.01,0.01,0.00,0.0,0.00,0.0,0.06,0.01,0.00,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.01,0.0,0.00,0.0,0.0,0.0,0.0
1,Taipei,Datong,25.065986,121.515514,0.0,0.00,0.00,0.0,0.00,0.01,0.01,0.04,0.0,0.00,0.0,0.02,0.01,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.0,0.0,0.03,0.0,0.00,0.00,0.

#### Let's confirm the new size

In [19]:
all_grouped.shape

(41, 206)

In [20]:
all_merged_data.shape

(41, 209)

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    # mark venue categories whcih do not exist in that neighborhood with 'NaN'
    for i in range(len(row_categories_sorted)):
        if row_categories_sorted[i]==0.0:
            row_categories_sorted.index.values[i]='NaN'
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [22]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = all_grouped['Neighborhood']

for ind in np.arange(all_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(all_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bali,Tunnel,History Museum,Boat or Ferry,Café,Noodle House,Hotel,Hot Spring,Harbor / Marina,NaN,NaN
1,Banqiao,Coffee Shop,Convenience Store,Shopping Mall,Asian Restaurant,Fast Food Restaurant,Restaurant,Multiplex,Stadium,Café,Train Station
2,Beitou,Hotel,Hot Spring,Convenience Store,Noodle House,Resort,Park,Café,Japanese Restaurant,Chinese Restaurant,Coffee Shop
3,Da'an,Café,Taiwanese Restaurant,Noodle House,Chinese Restaurant,Hotel,Bakery,Bubble Tea Shop,Sushi Restaurant,Japanese Restaurant,Pizza Place
4,Datong,Taiwanese Restaurant,Hotel,Chinese Restaurant,Dessert Shop,Japanese Restaurant,Hotpot Restaurant,Coffee Shop,Asian Restaurant,Market,Breakfast Spot


In [23]:
neighborhoods_venues_sorted.shape

(41, 11)

## 4. Cluster Neighborhoods

In [24]:
all_grouped_clustering = all_grouped.drop('Neighborhood', 1)
all_grouped_clustering.head()

,ATM,Airport Service,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bakery,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beijing Restaurant,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buddhist Temple,Building,Burger Joint,Bus Station,Bus Stop,Cable Car,Cafeteria,Café,Campground,Cantonese Restaurant,Cha Chaan Teng,Chinese Breakfast Place,Chinese Restaurant,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cultural Center,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donburi Restaurant,Donut Shop,Dumpling Restaurant,Duty-free Shop,Electronics Store,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Historic Site,History Museum,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jiangsu Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Lounge,Malay Restaurant,Market,Massage Studio,Metro Station,Mexican Restaurant,Miscellaneous Shop,Mongolian Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Night Market,Nightclub,Noodle House,Office,Other Great Outdoors,Other Nightlife,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Seafood Restaurant,Shanghai Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Skate Park,Snack Place,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Supermarket,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tea Room,Temple,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tourist Information Center,Toy / Game Store,Trail,Train,Train Station,Tunnel,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfall,Women's Store,Zoo,Zoo Exhibit
0,0.000000,0.0,0.000000,0.0,0.0,0.00,0.00,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.0,0.00,0.125,0.000000,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.125000,0.0,0.0,0.00,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.00,0.00,0.000000,0.00,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.00,0.125,0.000000,0.125000,0.0,0.0,0.0,0.125000,0.125000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.00,0.000000,0.0,0.0,0.00,0.0,0.0,0.00,0.000000,0.00,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.125000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.00,0.0,0.000000,0.00,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00,0.00,0.0,0.00,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.125,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0135

Run *k*-means to cluster the neighborhood into 5 clusters.

In [25]:
# set number of clusters
kclusters = 5

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(all_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 1, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [26]:
# add clustering labels and merge all_merged and neighborhoods_venues_sorted dataframes for data visualization later.
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

all_merged = geo_data

all_merged = all_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

all_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Taipei,Zhongzheng,25.047565,121.516946,0,Café,Hotel,Taiwanese Restaurant,Noodle House,Coffee Shop,Hostel,Asian Restaurant,Chinese Restaurant,Bakery,Hotpot Restaurant
1,Taipei,Datong,25.065986,121.515514,0,Taiwanese Restaurant,Hotel,Chinese Restaurant,Dessert Shop,Japanese Restaurant,Hotpot Restaurant,Coffee Shop,Asian Restaurant,Market,Breakfast Spot
2,Taipei,Zhongshan,25.064361,121.533468,0,Hotel,Chinese Restaurant,Taiwanese Restaurant,Café,Hotpot Restaurant,Japanese Restaurant,Dumpling Restaurant,Asian Restaurant,Park,Seafood Restaurant
3,Taipei,Songshan,25.049885,121.577272,0,Café,Noodle House,Department Store,Hotel,Chinese Restaurant,Hotpot Restaurant,Park,Coffee Shop,Taiwanese Restaurant,Bookstore
4,Taipei,Da'an,25.026515,121.534395,0,Café,Taiwanese Restaurant,Noodle House,Chinese Restaurant,Hotel,Bakery,Bubble Tea Shop,Sushi Restaurant,Japanese Restaurant,Pizza Place


Finally, let's visualize the resulting clusters

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(all_merged['Latitude'], all_merged['Longitude'], all_merged['Neighborhood'], all_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine and visualize each cluster and determine the discriminating venue categories that distinguish each cluster by viewing the top 10 venues for each neighborhood.

#### Cluster 0

In [28]:
cluster_0=all_merged.loc[all_merged['Cluster Labels'] == 0]
cluster_0

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Taipei,Zhongzheng,25.047565,121.516946,0,Café,Hotel,Taiwanese Restaurant,Noodle House,Coffee Shop,Hostel,Asian Restaurant,Chinese Restaurant,Bakery,Hotpot Restaurant
1,Taipei,Datong,25.065986,121.515514,0,Taiwanese Restaurant,Hotel,Chinese Restaurant,Dessert Shop,Japanese Restaurant,Hotpot Restaurant,Coffee Shop,Asian Restaurant,Market,Breakfast Spot
2,Taipei,Zhongshan,25.064361,121.533468,0,Hotel,Chinese Restaurant,Taiwanese Restaurant,Café,Hotpot Restaurant,Japanese Restaurant,Dumpling Restaurant,Asian Restaurant,Park,Seafood Restaurant
3,Taipei,Songshan,25.049885,121.577272,0,Café,Noodle House,Department Store,Hotel,Chinese Restaurant,Hotpot Restaurant,Park,Coffee Shop,Taiwanese Restaurant,Bookstore
4,Taipei,Da'an,25.026515,121.534395,0,Café,Taiwanese Restaurant,Noodle House,Chinese Restaurant,Hotel,Bakery,Bubble Tea Shop,Sushi Restaurant,Japanese Restaurant,Pizza Place
5,Taipei,Wanhua,25.031933,121.499332,0,Noodle House,Taiwanese Restaurant,Coffee Shop,Café,Hotel,Hostel,Dessert Shop,Park,Hotpot Restaurant,Ice Cream Shop
6,Taipei,Xinyi,25.037520,121.563680,0,Department Store,Café,Hotel,Dessert Shop,Cocktail Bar,Japanese Restaurant,Bookstore,Lounge,Noodle House,Chinese Restaurant
7,Taipei,Shilin,25.091840,121.524207,0,Convenience Store,Café,Taiwanese Restaurant,Coffee Shop,Breakfast Spot,Noodle House,Park,Chinese Restaurant,Tea Room,Dessert Shop
8,Taipei,Beitou,25.132419,121.501379,0,Hotel,Hot Spring,Convenience Store,Noodle House,Resort,Park,Café,Japanese Restaurant,Chinese Restaurant,Coffee Shop
12,New Taipei City,Banqiao,25.009670,121.459099,0,Coffee Shop,Convenience Store,Shopping Mall,Asian Restaurant,Fast Food Restaurant,Restaurant,Multiplex,Stadium,Café,Train Station


Let's visualize cluster 0:

In [29]:
# create map for cluster 0
map_cluster_0 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lon, poi, cluster in zip(cluster_0['Latitude'], cluster_0['Longitude'], cluster_0['Neighborhood'], cluster_0['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster_0)
       
map_cluster_0

#### Cluster 1

In [30]:
cluster_1=all_merged.loc[all_merged['Cluster Labels'] == 1]
cluster_1

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,New Taipei City,Gongliao,25.022043,121.910082,1,Train Station,Campground,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's visualize cluster 1:

In [31]:
# create map for cluster 1
map_cluster_1 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lon, poi, cluster in zip(cluster_1['Latitude'], cluster_1['Longitude'], cluster_1['Neighborhood'], cluster_1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster_1)
       
map_cluster_1

#### Cluster 2

In [32]:
cluster_2=all_merged.loc[all_merged['Cluster Labels'] == 2]
cluster_2

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,Taipei,Neihu,25.069664,121.588998,2,Convenience Store,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Noodle House,Bakery,Supermarket,Chinese Restaurant,Fast Food Restaurant
10,Taipei,Nangang,25.054578,121.606600,2,Convenience Store,Coffee Shop,Fast Food Restaurant,Noodle House,Japanese Restaurant,Café,Restaurant,Park,Chinese Restaurant,Thai Restaurant
11,Taipei,Wenshan,24.989786,121.570458,2,Convenience Store,Coffee Shop,Exhibit,Zoo Exhibit,Café,Breakfast Spot,Farmers Market,Asian Restaurant,Fast Food Restaurant,Chinese Restaurant
14,New Taipei City,Zhonghe,24.999397,121.498980,2,Coffee Shop,Convenience Store,Japanese Restaurant,Park,Dessert Shop,Night Market,Noodle House,Café,Taiwanese Restaurant,Tea Room
16,New Taipei City,Tucheng,24.972201,121.443348,2,Convenience Store,Bus Stop,Chinese Restaurant,Bubble Tea Shop,American Restaurant,Fast Food Restaurant,Supermarket,Shopping Mall,Italian Restaurant,Café
18,New Taipei City,Sanxia,24.934339,121.368905,2,Japanese Restaurant,Fast Food Restaurant,Dumpling Restaurant,Convenience Store,Bakery,Coffee Shop,Dessert Shop,Noodle House,Chinese Restaurant,Dim Sum Restaurant
21,New Taipei City,Luzhou,25.084923,121.473700,2,Convenience Store,Coffee Shop,Shopping Mall,Sushi Restaurant,Japanese Restaurant,Bakery,Chinese Restaurant,Night Market,Movie Theater,Miscellaneous Shop
23,New Taipei City,Taishan,25.058864,121.430811,2,Convenience Store,Bakery,Market,Dessert Shop,Mountain,Burger Joint,NaN,NaN,NaN,NaN
24,New Taipei City,Linkou,25.077531,121.391602,2,Convenience Store,Japanese Restaurant,Coffee Shop,Department Store,Skate Park,Italian Restaurant,Breakfast Spot,NaN,NaN,NaN
29,New Taipei City,Jinshan,25.221883,121.636427,2,Convenience Store,Dessert Shop,Taiwanese Restaurant,Asian Restaurant,Market,Steakhouse,Surf Spot,Dumpling Restaurant,Resort,Café


Let's visualize cluster 2:

In [33]:
# create map for cluster 2
map_cluster_2 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lon, poi, cluster in zip(cluster_2['Latitude'], cluster_2['Longitude'], cluster_2['Neighborhood'], cluster_2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster_2)
       
map_cluster_2

#### Cluster 3

In [34]:
cluster_3=all_merged.loc[all_merged['Cluster Labels'] == 3]
cluster_3

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,New Taipei City,Wugu,25.082743,121.438156,3,Bakery,Baseball Field,Convenience Store,Fast Food Restaurant,Lake,NaN,NaN,NaN,NaN,NaN


Let's visualize cluster 3:

In [35]:
# create map for cluster 3
map_cluster_3 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lon, poi, cluster in zip(cluster_3['Latitude'], cluster_3['Longitude'], cluster_3['Neighborhood'], cluster_3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster_3)
       
map_cluster_3

#### Cluster 4

In [36]:
cluster_4=all_merged.loc[all_merged['Cluster Labels'] == 4]
cluster_4

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,New Taipei City,Wanli,25.181234,121.688687,4,Seafood Restaurant,Beach,Harbor / Marina,Resort,Market,Park,Bakery,NaN,NaN,NaN


Let's visualize cluster 4:

In [37]:
# create map for cluster 4
map_cluster_4 = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lon, poi, cluster in zip(cluster_4['Latitude'], cluster_4['Longitude'], cluster_4['Neighborhood'], cluster_4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_cluster_4)
       
map_cluster_4